In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Arrhythmia/GSE53622'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"LncRNA profile study reveals a three-lncRNA signature associated with the survival of esophageal squamous cell carcinoma patients (independent set)"
!Series_summary	"Objective: Globally, esophageal cancer is among the most deadly cancer forms. Long non-coding RNAs (lncRNA) are frequently found to have important regulatory roles. We aim to assess the lncRNA expression profile of esophageal squamous cell carcinoma (ESCC) and identify prognosis related lncRNAs. Design: LncRNA expression profiles were studied by microarray in paired tumor and normal tissues from 119 ESCC patients, and validated by qRT-PCR. The 119 patients were subsequently divided randomly into training (n=60) and test (n=59) groups. A prognostic signature was developed from the training group using a random forest supervised classification algorithm and a nearest shrunken centroid algorithm, and validated in test group and further in an independent cohort (n=60). The independence of

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 10
age_row = 1
gender_row = 2

def convert_trait(value):
    val = value.split(':')[-1].strip().lower()
    if val in ['yes', 'no']:
        return 1 if val == 'yes' else 0
    return None

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(':')[-1].strip().lower()
    if val in ['male', 'female']:
        return 1 if val == 'male' else 0
    return None

save_cohort_info('GSE53622', './preprocessed/Arrhythmia/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Arrhythmia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Arrhythmia/trait_data/GSE53622.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1296956': [0.0, 66.4602739726027, 0.0], 'GSM1296957': [0.0, 66.4602739726027, 0.0], 'GSM1296958': [1.0, 64.013698630137, 1.0], 'GSM1296959': [1.0, 64.013698630137, 1.0], 'GSM1296960': [0.0, 50.9123287671233, 1.0], 'GSM1296961': [0.0, 50.9123287671233, 1.0], 'GSM1296962': [1.0, 46.3287671232877, 1.0], 'GSM1296963': [1.0, 46.3287671232877, 1.0], 'GSM1296964': [0.0, 53.9972602739726, 1.0], 'GSM1296965': [0.0, 53.9972602739726, 1.0], 'GSM1296966': [1.0, 67.8438356164384, 0.0], 'GSM1296967': [1.0, 67.8438356164384, 0.0], 'GSM1296968': [0.0, 64.8794520547945, 1.0], 'GSM1296969': [0.0, 64.8794520547945, 1.0], 'GSM1296970': [0.0, 45.2219178082192, 1.0], 'GSM1296971': [0.0, 45.2219178082192, 1.0], 'GSM1296972': [1.0, 54.4794520547945, 1.0], 'GSM1296973': [1.0, 54.4794520547945, 1.0], 'GSM1296974': [0.0, 56.2328767123288, 1.0], 'GSM1296975': [0.0, 56.2328767123288, 1.0], 'GSM1296976': [0.0, 57.0986301369863, 1.0], 'GSM1296977': [0.0, 57.0986301369863, 1.0], 'GSM1296978': [0.0, 44.6630136986

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '24', '25', '26', '27', '28', '29', '31', '33', '34', '35',
       '36', '39', '40', '41', '42', '43', '45', '46'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['206', '1837', '3355', '1510', '2286'], 'COL': ['532', '527', '523', '528', '526'], 'ROW': [269.0, 68.0, 91.0, 42.0, 189.0], 'SPOT_ID': ['(-)3xSLv1', '(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107'], 'Control Type': ['neg', 'pos', 'pos', 'pos', 'pos'], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the keys in the annotation dictionary:
identifier_key = 'ID'
gene_symbol_key = 'SPOT_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Arrhythmia/gene_data/GSE53622.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Arrhythmia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE53622', './preprocessed/Arrhythmia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Arrhythmia/GSE53622.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Arrhythmia', the least common label is '1.0' with 32 occurrences. This represents 26.67% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.933561643835624
  50% (Median): 60.23972602739725
  75%: 66.3184931506849
Min: 39.5397260273973
Max: 81.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 24 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

